# Escrow Using Marlowe Runtime\'s REST API

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

The escrow contract example is a simple Marlowe contract where a seller offers merchandise for sale and a buyer purchases it; if the buyer is not satisfied with the merchandise, they may dispute the purchase, in which case a mediator may rule for the buyer or seller.

[A video works through this Jupyter notebook.](https://youtu.be/E8m-PKbS9TI)

You can ask questions about Marlowe in [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) or post problems with this lesson to [the issues list for the Marlowe Starter Kit github repository](https://github.com/input-output-hk/marlowe-starter-kit/issues).

In this demonsration we use Marlowe Runtime\'s REST API, served via `marlowe-web-server`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use role tokens and we have Marlowe Runtime mint them.

In [Marlowe Playground](https://play.marlowe.iohk.io//), the contract looks like this in Blockly format.

![Marlowe contract for escrow](images/escrow-playground.png)

In Marlowe format it appears as
```
When [
  (Case
     (Deposit Role "Seller" Role "Buyer"
        (Token "" "")
        (ConstantParam "Price"))
     (When [
           (Case
              (Choice
                 (ChoiceId "Everything is alright" Role "Buyer") [
                 (Bound 0 0)]) Close)
           ,
           (Case
              (Choice
                 (ChoiceId "Report problem" Role "Buyer") [
                 (Bound 1 1)])
              (Pay Role "Seller"
                 (Account Role "Buyer")
                 (Token "" "")
                 (ConstantParam "Price")
                 (When [
                       (Case
                          (Choice
                             (ChoiceId "Confirm problem" Role "Seller") [
                             (Bound 1 1)]) Close)
                       ,
                       (Case
                          (Choice
                             (ChoiceId "Dispute problem" Role "Seller") [
                             (Bound 0 0)])
                          (When [
                                (Case
                                   (Choice
                                      (ChoiceId "Dismiss claim" Role "Mediator") [
                                      (Bound 0 0)])
                                   (Pay Role "Buyer"
                                      (Party Role "Seller")
                                      (Token "" "")
                                      (ConstantParam "Price") Close))
                                ,
                                (Case
                                   (Choice
                                      (ChoiceId "Confirm problem" Role "Mediator") [
                                      (Bound 1 1)]) Close)]
                                      (TimeParam "Mediation deadline") Close))] 
                                      (TimeParam "Complaint response deadline") Close)))] 
                                      (TimeParam "Complaint deadline") Close))] 
                                      (TimeParam "Payment deadline") Close
```

The flow chart below shows the possible execution paths of the escrow contract. This example demonstrates the shaded path.

![Flow chart for escrow contract, with the "dismiss claim" execution path highlighted](images/04-dismiss-claim.svg)

## Preliminaries

See [Lesson 0. Preliminaries](00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.
- `MARLOWE_RT_WEBSERVER_HOST`: IP address of the Marlowe Runtime web server.
- `MARLOWE_RT_WEBSERVER_PORT`: Port number for the Marlowe Runtime web server.

It also assumes that the parties have addresses, signing keys, and funds.
- Seller
    - [keys/lender.address](keys/lender.address): Cardano address for the seller
    - [keys/lender.skey](keys/lender.skey): location of signing key file for the seller
- Buyer
    - [keys/borrower.address](keys/borrower.address): Cardano address for the buyer
    - [keys/borrower.skey](keys/borrower.skey): location of signing key file for the buyer
- Mediator
    - [keys/mediator.address](keys/mediator.address): Cardano address for the mediator
    - [keys/mediator.skey](keys/mediator.skey): location of signing key file for the mediator

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [ ]:
if [[ -z "$MARLOWE_RT_WEBSERVER_PORT" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

  # Only required for Marlowe Runtime REST API.
  export MARLOWE_RT_WEBSERVER_HOST="127.0.0.1"
  export MARLOWE_RT_WEBSERVER_PORT=3780

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

MARLOWE_RT_WEBSERVER_URL="http://$MARLOWE_RT_WEBSERVER_HOST":"$MARLOWE_RT_WEBSERVER_PORT"

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"
echo "MARLOWE_RT_WEBSERVER_HOST = $MARLOWE_RT_WEBSERVER_HOST"
echo "MARLOWE_RT_WEBSERVER_PORT = $MARLOWE_RT_WEBSERVER_PORT"
echo "MARLOWE_RT_WEBSERVER_URL = $MARLOWE_RT_WEBSERVER_URL"

Note the test network magic number:
- `preprod` = 1
- `preview` = 2

### Seller address and funds

Check that an address and key has been created for the seller. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [ ]:
SELLER_SKEY=keys/lender.skey
SELLER_ADDR=$(cat keys/lender.address)
echo "SELLER_ADDR = $SELLER_ADDR"

Check that the seller has at least one hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SELLER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/address/"$SELLER_ADDR"

### Buyer address and funds

Check that an address and key has been created for the buyer. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [ ]:
BUYER_SKEY=keys/borrower.skey
BUYER_ADDR=$(cat keys/borrower.address)
echo "BUYER_ADDR = $BUYER_ADDR"

Check that the buyer has at least one hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BUYER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/address/"$BUYER_ADDR"

### Mediator address and funds

Check that an address and key has been created for the mediator. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [ ]:
MEDIATOR_SKEY=keys/mediator.skey
MEDIATOR_ADDR=$(cat keys/mediator.address)
echo "MEDIATOR_ADDR = $MEDIATOR_ADDR"

Check that the mediator has at least one hundred ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MEDIATOR_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/address/"$MEDIATOR_ADDR"

## Design the contract

The escrow contract can be downloaded from the [Marlowe Playground](https://play.marlowe.iohk.io//) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Set the purchase prices to 75 ada.

In [ ]:
ADA=1000000  # 1 ada = 1,000,000 lovelace
PRICE=$((75 * ADA))
echo "PRICE = $PRICE lovelace"

On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [ ]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

Later in the example we will need some constants for converting times.

In [ ]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

### *Alternative 1:* Use Marlowe Playground to design the contract

If you want to create the contract in Marlowe Playground, do the following:

1. Visit https://play.marlowe.iohk.io// in a web browser.
2. Select "Open an Example".
3. Select "Marlowe" or "Blockly" under "Escrow".
4. Select "Send to Simulator".
5. Set the "Payment deadline" to one hour into the future.
6. Set the "Complaint deadline" to two hours into the future
7. Set the "Complaint response deadline" to three hours into the future.
8. Set the "Mediation deadline" to four hours into the future.
9. Set the "Price" to 75 ada.
10. Select "Download as JSON", set the file name to "escrow-contract.json", and store the file in this folder, namely [marlowe-starter-kit/](.). Note that most interfaces for Jupyter notebooks allow one to drag and drop a file into their explorer panel.

***Be careful setting the deadlines because, if these mistakenly are in the past or in the too-near future, then parts of the contract will time out and not operate as described in this tutorial.***

![Setting parameters for the escrow bond contract in Marlowe Playground](images/escrow-simulation.png)

### *Alternative 2:* Use Marlowe CLI to generate the contract

Below we generate the contract using Marlowe CLI.

First find the current time, measured in [POSIX milliseconds](https://en.wikipedia.org/wiki/Unix_time).

In [ ]:
NOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NOW" POSIX milliseconds = "`date -d @$((NOW / SECOND))`"

The contract has four deadlines. For convenience in this example, set the deadlines to the near future.

In [ ]:
PAYMENT_DEADLINE=$((NOW+1*HOUR))    # The payment deadline, one hour from now.
COMPLAINT_DEADLINE=$((NOW+2*HOUR))  # The complaint deadline, two hours from now.
DISPUTE_DEADLINE=$((NOW+3*HOUR))    # The dispute deadline, three hours from now.
MEDIATION_DEADLINE=$((NOW+4*HOUR))  # The mediation deadline, four hours from now.

echo PAYMENT_DEADLINE = "$PAYMENT_DEADLINE" POSIX milliseconds = "`date -d @$((PAYMENT_DEADLINE / SECOND))`"
echo COMPLAINT_DEADLINE = "$COMPLAINT_DEADLINE" POSIX milliseconds = "`date -d @$((COMPLAINT_DEADLINE / SECOND))`"
echo DISPUTE_DEADLINE = "$DISPUTE_DEADLINE" POSIX milliseconds = "`date -d @$((DISPUTE_DEADLINE / SECOND))`"
echo MEDIATION_DEADLINE = "$MEDIATION_DEADLINE" POSIX milliseconds = "`date -d @$((MEDIATION_DEADLINE / SECOND))`"

Now create the JSON file for the contract, `zcb-contract.json`.

In [ ]:
marlowe-cli template escrow \
  --minimum-ada "$MIN_LOVELACE" \
  --price "$PRICE" \
  --seller Seller \
  --buyer Buyer \
  --mediator Mediator \
  --payment-deadline "$PAYMENT_DEADLINE" \
  --complaint-deadline "$COMPLAINT_DEADLINE" \
  --dispute-deadline "$DISPUTE_DEADLINE" \
  --mediation-deadline "$MEDIATION_DEADLINE" \
  --out-contract-file escrow-contract.json \
  --out-state-file /dev/null

The various command-line options are described by the help system.

In [ ]:
marlowe-cli template escrow --help

## Examine the contract

View the contract file as YAML.

In [ ]:
json2yaml escrow-contract.json

### \[Optional, but recommended\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe.iohk.io//) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](01-runtime-cli.ipynb) for an example of performing step 6.

## Transaction 1: Mediator Creates Escrow Contract with Initial ADA

A `HTTP` `POST` request to Marlowe Runtime\'s `/contracts` endpoint will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

First we create the JSON body of the request to build the creation transaction.

In [ ]:
yaml2json << EOI > request-1.json
version: v1
contract: `cat escrow-contract.json`
roles:
  Seller: "$SELLER_ADDR"
  Buyer: "$BUYER_ADDR"
  Mediator: "$MEDIATOR_ADDR"
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: {}
EOI
cat request-1.json

Next we post the request and view the response.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $MEDIATOR_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

The identifier for the contract is embedded in the response.

In [ ]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

The CBOR serialization (in text-envelope format) is also embedded in the response.

In [ ]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [ ]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$MEDIATOR_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

One can view the transaction on a Cardano explorer and see that the contract has been created and the parties have received their role tokens. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_1?tab=utxo"

In particular, we see that the Marlowe contract holds the 2 ada that was set as `MINIMUM_LOVELACE`.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$CONTRACT_ID"

One can see that the seller, buyer, and mediator have received their role tokens. Note that `4c656e646572 = Seller`, `4275796572 = Buyer`, and `4d65646961746f72 = Mediator` in hexadecimal notation.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SELLER_ADDR"

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BUYER_ADDR"

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MEDIATOR_ADDR"

## View the details of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL = $CONTRACT_URL"

In [ ]:
curl -sS "$CONTRACT_URL" | json2yaml

## Transaction 2: Buyer Deposits Funds into Seller’s Account

The buyer deposits their 75 ada into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint. The buyer is providing the funding for and receiving the change from this transaction, so we provide their address.

The deposit is represented as JSON input to the contract. The `marlowe-cli input deposit` tool conveniently formats the correct JSON for a deposit.

In [ ]:
marlowe-cli input deposit --help

In [ ]:
marlowe-cli input deposit \
  --deposit-party Buyer \
  --deposit-account Seller \
  --deposit-amount "$PRICE" \
  --out-file input-2.json
json2yaml input-2.json

In [ ]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json

Next we post the request and store the response.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BUYER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

In [ ]:
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$BUYER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_2?tab=utxo"

One can see that the buyer has approximately 75 ada less than originally.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BUYER_ADDR"

The Marlowe contract still has the 2 ada from its creation and an additional 75 ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_2" | json2yaml

## Transaction 3: The Buyer Reports That There is a Problem

The buyer chooses to report a problem with the merchandise.

The choice is represented as JSON input to the contract. The `marlowe-cli input choose` tool conveniently formats the correct JSON for a choice.

In [ ]:
marlowe-cli input choose --help

In [ ]:
marlowe-cli input choose \
  --choice-name "Report problem" \
  --choice-party Buyer \
  --choice-number 1 \
  --out-file input-3.json
json2yaml input-3.json

In [ ]:
yaml2json << EOI > request-3.json
version: v1
inputs: [$(cat input-3.json)]
metadata: {}
tags: {}
EOI
cat request-3.json

Next we post the request and store the response.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BUYER_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-3.json > tx-3.unsigned

In [ ]:
TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$BUYER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

One can see that the buyer still has approximately 75 ada less than originally.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BUYER_ADDR"

The Marlowe contract still has the 77 ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_3#1"

## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_3" | json2yaml

## Transaction 4: The Seller Disputes that There is a Problem

Now the seller chooses to dispute that there is a problem with the merchandise.

In [ ]:
marlowe-cli input choose \
  --choice-name "Dispute problem" \
  --choice-party Seller \
  --choice-number 0 \
  --out-file input-4.json
json2yaml input-4.json

In [ ]:
yaml2json << EOI > request-4.json
version: v1
inputs: [$(cat input-4.json)]
metadata: {}
tags: {}
EOI
cat request-4.json

Next we post the request and store the response.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SELLER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

In [ ]:
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$SELLER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_4?tab=utxo"

One can see that the seller still has approximately their original balance.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SELLER_ADDR"

The Marlowe contract still has the 77 ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_4#1"

## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_4" | json2yaml

## Transaction 5: The Mediator Dismisses the Claim

The mediator rules in favor or the seller and dismisses the buyer's claim.

In [ ]:
marlowe-cli input choose \
  --choice-name "Dismiss claim" \
  --choice-party Mediator \
  --choice-number 0 \
  --out-file input-5.json
json2yaml input-5.json

In [ ]:
yaml2json << EOI > request-5.json
version: v1
inputs: [$(cat input-5.json)]
metadata: {}
tags: {}
EOI
cat request-5.json

Next we post the request and store the response.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $MEDIATOR_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS
json2yaml response-5.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-5.json > tx-5.unsigned

In [ ]:
TX_5=$(
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$MEDIATOR_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_5?tab=utxo"

One can see that the mediator still has approximately their original balance, which includes the 2 ada they just received, refunding the 2 ada they used to create the contract.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MEDIATOR_ADDR"

The Marlowe contract is closed, but the role-payout address has the 75 ada for the benefit of the seller.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_5#2"

## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [ ]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_5" | json2yaml

## Transaction 6: The Seller Withdraws Their Funds

The price of 75 ada is held at Marlowe's role-payout address for the benefit of the seller. The seller can withdraw these funds at any time. The contract ID and role name are included in the request body for a withdrawal.

In [ ]:
yaml2json << EOI > request-6.json
contractId: "$CONTRACT_ID"
role: Seller
EOI
cat request-6.json

Next we post the request and store the response.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SELLER_ADDR" \
  -d @request-6.json \
  -o response-6.json \
  -sS
json2yaml response-6.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-6.json > tx-6.unsigned

In [ ]:
TX_6=$(
marlowe-cli transaction submit \
  --tx-body-file tx-6.unsigned \
  --required-signer "$SELLER_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_6 = $TX_6"

On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "$EXPLORER_URL"/transaction/"$TX_6?tab=utxo"

The seller now has about an additional 75 ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SELLER_ADDR"

## View the withdrawal

Marlowe Runtime\'s `HTTP` `GET` endpoint `/withdrawals/{transactionId}` can fetch a withdrawal from the blockchain and return information about it.

In [ ]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL"/withdrawals/"$TX_6" | json2yaml